In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1) # passes input to layer 1
        self.fc2 = nn.Linear(h1, h2) # passes layer 1 to layer 2
        self.out = nn.Linear(h2, out_features) # passes layer 2 to output

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

In [ ]:
# Pick manual seed for randomization
torch.manual_seed(41)
# Create instance of model
model = Model()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
df = pd.read_csv(url)

In [ ]:
# Change last column from strings to integers
df['variety'] = df['variety'].replace('Setosa', 0.0)
df['variety'] = df['variety'].replace('Versicolor', 1.0)
df['variety'] = df['variety'].replace('Virginica', 2.0)


In [ ]:
# Train Test Split!  Set X, y
x = df.drop('variety', axis=1)
y = df['variety']


In [ ]:
# Convert these to numpy arrays
x = x.values
y = y.values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=41)

In [ ]:
# Convert X features to float tensors
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)

In [ ]:
# Convert y labels to tensors long
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
# Measure error
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.015)

In [ ]:
# Train model
epochs = 100
losses = []
for i in range(epochs):
    y_pred = model.forward(x_train)  # Send data forward to get predicted results

    # Measure loss
    loss = criterion(y_pred, y_train)  # Predicted values vs trained values
    losses.append(loss.item())  # Convert loss to a Python float and append to list

    # Print every 10 epochs
    if i % 10 == 0:
        print(f"Epoch count: {i} loss: {loss.item()}")

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

In [ ]:
# Plot results
plt.plot(range(epochs), losses)
plt.ylabel("loss/error")
plt.xlabel("Epoch")

In [ ]:
# Evaluate model on test data set
with torch.no_grad(): #Turn off back propogation
    y_eval = model.forward(x_test) # x_test - features from test set - y_test - predictions
    loss = criterion(y_eval, y_test) # find error
    print(loss)

In [ ]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(x_test):
    y_val = model.forward(data)

    if y_test[i] == 0:
      x = "Setosa"
    elif y_test[i] == 1:
      x = 'Versicolor'
    else:
      x = 'Virginica'


    # Will tell us what type of flower class our network thinks it is
    print(f"{i+1}.)  {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}")

    # Correct or not
    if y_val.argmax().item() == y_test[i]:
      correct +=1

print(f"{correct} correct!")

In [52]:
# Create new datapoint

new_iris = torch.tensor([4.9, 3.0, 1.4, 0.2])

In [53]:
# Feed new datapoint to model 

with torch.no_grad():
  pred_tensor = (model(new_iris))
  pred_index = torch.argmax(pred_tensor)

# Derive prediction

if pred_index == 0:
  pred_iris = "Setosa"

elif pred_index == 1:
  pred_iris = "Versicolor"

else:
  pred_iris = "Virginica"

print(f"Predicted flower: {pred_iris}")
print(pred_tensor)

Predicted flower: Setosa
tensor([ 5.0861, -2.5376, -3.4450])
